In [1]:
# Importing libraries
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import os   
import h5py
from tslearn.metrics import dtw
from sklearn.preprocessing import MinMaxScaler
sys.path.append('../')
from utils.data_utils import *
from utils.paths import CLEAN_DATA_DIR

from timezonefinder import TimezoneFinder
import time

import warnings
warnings.filterwarnings('ignore')



In [2]:

# Set working directory
os.chdir(r"..") # should be the git repo root directory
print("Current working directory: " + os.getcwd())
assert os.getcwd()[-8:] == "WattCast", "Current working directory is not the git repo root directory"

raw_data_path = os.path.join(os.getcwd(),'data','raw_data')
if not os.path.exists(raw_data_path):
    os.makedirs(raw_data_path)
    print("Created directory: " + raw_data_path)

save_path = os.path.join(os.getcwd(),'data','clean_data')
if not os.path.exists(save_path):
    os.makedirs(save_path)
    print("Created directory: " + save_path)

Current working directory: /Users/nikolaushouben/Desktop/WattCast


# <u>Raw data import, visualization & cleaning<u/>


Criteria for Data Collection: 

* at least 1 year of training data and 1 summer and 1 winter month of testing data are available
* at least 1 hour resolution (higher resolution preferred)
* less than 1% missing values or invalid measurements


Structure of Blocks:

1. Each block starts with the Name of the dataset and its download link, alternatively you can follow this [link](https://www.dropbox.com/sh/fvx3wune2qg2x43/AADP4F3UwIqrS9tYnN6mTob5a?dl=0) to download the raw and cleaned data directory from our gdrive.
2. The overview states for which aggregation level the data was used. For example, while (1) BA dataset is only used on the county level, the (2) Substation dataset from Germany is used for both the neighborhood level in an aggregated form and on the household level in its disaggregated form.
3. Power and Weather data are then imported and cleaned, wherever necessary.
4. Each block also provides an interactive visualization of the timeseries, often resampled to a lower temporal resolution to make it run faster.
5. The block ends with spliting* the data into one train (1 year) and two test datasets (one in summer and winter); once more 60 minute and, if available, in 15 minute resolution

a) <u>Note</u> on the train test splits: Not all data was recorded in the same temporal resolution and during the same time. However, training and testing set lengths were kept consistent to ensure a fair comparison of algorithms across scales. 

b) <u>Note</u> on the timezones: Data comes in various timezones and are always processed to match the local timezone. This helps with interepretability of results due to known patters of human behaviour.

## &#x2460; 'EIA Cleaned Hourly Electricity Demand Data' - Balancing Authority (United States of America)
[![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.3690240.svg)](https://doi.org/10.5281/zenodo.3690240)

Used for: 
* 1_county
----------
* Duration: 3 years
* Resolution: 60 minutes
* Power Level: 1-50 GW

As in https://doi.org/10.1016/j.adapen.2021.100025, We used the data from three BAs: the Los Angeles Department of Water and Power (LADWP), Balance Authority of Northern California (BANC), and New York Independent System Operator (NYISO),

### Data Prep

Load

In [ ]:
# set this path to the directory where the data is stored

dir_path = os.path.join(raw_data_path,'balancing_authorities_US','power','{}.csv')


dir_path_weather = os.path.join(raw_data_path,'balancing_authorities_US','weather','{}.csv')
county_acronnyms = {"LDWP":"Los_Angeles", "BANC":"Sacramento", "NYIS":"New_York"}

county_coordinates = {"Los_Angeles":(34.08851, -118.234216), "Sacramento":(38.5815719, -121.4943996), "New_York":(40.730610, -73.935242)}

unit = "GW" # unit of the demand data
temp_resolutions = [60] # minutes, this dataset is only available in hourly resolution
spatial_scale = "1_county" # spatial scale of the data

# Determined visually to meet criteria mentioned above
train_begin = "2016-01-01"
train_end = "2017-01-01"
# Winter
val_begin = "2017-01-01"
val_end = "2017-02-01"
#Summer
test_begin = "2017-08-15" # select a period with a heat wave
test_end = "2017-09-15"  # select a period with a heat wave

store = pd.HDFStore(os.path.join(save_path, f"{spatial_scale}.h5"), mode='w')
for temp_resolution in temp_resolutions:
    dfs = {}
    for key, value in county_acronnyms.items():
        
        lat, lng = county_coordinates[value]
        tf = TimezoneFinder()
        tz = tf.timezone_at(lng=lng, lat=lat)
        # power
        df = pd.read_csv(dir_path.format(key), index_col=0,parse_dates=True)
        df = df.loc[df["category"] == "OKAY"]['cleaned demand (MW)'].to_frame(f'{value}_demand_{unit}')/1e3 # only keep the demand data that is labeled as "OKAY" see paper for more details
        df = df.tz_localize('UTC').tz_convert(tz).tz_localize(None)
        df = standardize_format(df, 'load', temp_resolution, key, unit)
        df = remove_non_positive_values(df, set_nan=True)
        train, val, test = split_train_val_test_datasets(df, train_begin, train_end, val_begin, val_end, test_begin, test_end)
        dfs[value] = df
        store.put(f'{value}/{temp_resolution}min/train_target', train, format='table')
        store.put(f'{value}/{temp_resolution}min/val_target', val, format='table')
        store.put(f'{value}/{temp_resolution}min/test_target', test, format='table')

        # weather

        start_date = df.index[0].strftime("%Y-%m-%d")
        end_date = df.index[-1].strftime("%Y-%m-%d")
        df_weather = get_weather_data(lat, lng, start_date, end_date, variables=['temperature_2m'], keep_UTC=False).tz_localize(None)
        df_weather = standardize_format(df_weather, 'temperature', temp_resolution, value, "C")
        df_weather = df_weather.reindex(df.index).dropna() 
        dfs[value + 'weather'] = df_weather
        train_weather, val_weather, test_weather = split_train_val_test_datasets(df_weather, train_begin, train_end, val_begin, val_end, test_begin, test_end)

        store.put(f'{value}/{temp_resolution}min/train_cov', train_weather, format='table')
        store.put(f'{value}/{temp_resolution}min/val_cov', val_weather, format='table')
        store.put(f'{value}/{temp_resolution}min/test_cov', test_weather, format='table')

        
store.close()
# Merge all the dataframes into one
df_county = pd.concat(dfs.values(), axis=1)
df_county.columns = dfs.keys()
df_county.head()

### Visualization

In [ ]:

location = 'Sacramento'
df = df_county

fig = go.Figure()
# Add the ground truth data to the left axis
fig.add_trace(go.Scatter(y=df[location], x=df.index, name = location,  yaxis="y1"))
fig.add_trace(go.Scatter(y=df[location+'weather'], x=df.index, name = 'Temp',  yaxis="y2"))

fig.update_layout(
    xaxis=dict(title="Time"),
    yaxis=dict(title=f"Power", side="left"),
    yaxis2=dict(title="Temperature [°C]", overlaying="y", side="right"),
)

### Statistics

In [ ]:
calculate_stats_and_plot_hist(df_county[[col for col in df_county.columns if not 'weather' in col]])

## &#x2461; 'Electricity Load Diagrams' - Substation data (Portugal)
[![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.3690240.svg)](https://doi.org/10.24432/C58C86)

Used for: 

* 2_town
* 3_village

----------
* Duration: 4 years
* Resolution: 15 minutes
* Power Level: 0.05-50 MW (this broad range allows using this dataset for the two spatial scales: town & village)
* Timestamp: All time labels of the power data report to Portuguese hour (see https://archive-beta.ics.uci.edu/dataset/321/electricityloaddiagrams20112014)


Weather data is accessed through the https://open-meteo.com/en/docs API.

### Data Prep

In [ ]:
dir_path = os.path.join(raw_data_path,'substation_PT','LD2011_2014.txt')
temp_resolutions = [15, 60] # minutes


# Determined visually to meet criteria mentioned above
train_begin = "2011-01-01"
train_end = "2012-01-01"
# Winter
val_begin = "2012-01-01"
val_end = "2012-02-01"
#Summer
test_begin = "2012-06-15" # select a period with a heat wave
test_end = "2012-07-15"


#data import
df_sub = pd.read_csv(dir_path, sep=';', decimal = ",", index_col=0,parse_dates=True) 


### 2_town 

In [ ]:
# 2_town
spatial_scale = "2_town" # spatial scale of the data
lat = 39.399872 # TODO: get the lat and lng for the town
lng = -8.224454
unit = "MW" # unit of the demand data

#select the three columns with the highest median demand
df_sub_town = df_sub.loc[:, df_sub.median().sort_values(ascending=False).index].iloc[:, 1:4] / 1e3  # This data is in kW, we want to look at it in MW # the first location is missing the data for 2011 -> 1:4
town_acronnyms = {col: f'town_{i}' for i, col in enumerate(df_sub_town.columns)}


store = pd.HDFStore(os.path.join(save_path, f"{spatial_scale}.h5"), mode='w')
for temp_resolution in temp_resolutions:
# save the data for the towns and villages
    dfs = {}
    for key, value in town_acronnyms.items():

        # power
        df = df_sub_town[key].to_frame(f'{value}_demand_{unit}')
        df = standardize_format(df, 'load', temp_resolution, value, unit)
        df = remove_non_positive_values(df, set_nan=True)
        train, val, test = split_train_val_test_datasets(df, train_begin, train_end, val_begin, val_end, test_begin, test_end)
        dfs[value] = df
        store.put(f'{value}/{temp_resolution}min/train_target', train, format='table')
        store.put(f'{value}/{temp_resolution}min/val_target', val, format='table')
        store.put(f'{value}/{temp_resolution}min/test_target', test, format='table')

        start_date = df.index[0].strftime("%Y-%m-%d")
        end_date = df.index[-1].strftime("%Y-%m-%d")
        df_weather = get_weather_data(lat, lng, start_date, end_date, variables=['temperature_2m'], keep_UTC=False).tz_localize(None)
        df_weather = standardize_format(df_weather, 'temperature', temp_resolution, value, "C")
        df_weather = df_weather.reindex(df.index).dropna() 
        dfs[value + 'weather'] = df_weather
        train_weather, val_weather, test_weather = split_train_val_test_datasets(df_weather, train_begin, train_end, val_begin, val_end, test_begin, test_end)

        store.put(f'{value}/{temp_resolution}min/train_cov', train_weather, format='table')
        store.put(f'{value}/{temp_resolution}min/val_cov', val_weather, format='table')
        store.put(f'{value}/{temp_resolution}min/test_cov', test_weather, format='table')

store.close()      

# Merge all the dataframes into one
df_town = pd.concat(dfs.values(), axis=1)
df_town.columns = dfs.keys()



Visualization

In [ ]:
location = 'town_0'

df= df_town

fig = go.Figure()
# Add the ground truth data to the left axis
fig.add_trace(go.Scatter(y=df[location], x=df.index, name = location,  yaxis="y1"))
fig.add_trace(go.Scatter(y=df[location+'weather'], x=df.index, name = 'Temp',  yaxis="y2"))

fig.update_layout(
    xaxis=dict(title="Time"),
    yaxis=dict(title=f"Power", side="left"),
    yaxis2=dict(title="Temperature [°C]", overlaying="y", side="right"),
)

Statistics

In [ ]:
calculate_stats_and_plot_hist(df_town[[col for col in df_town.columns if not 'weather' in col]])

### 3_village 

In [ ]:
# 3_village
spatial_scale = "3_village" # spatial scale of the data
lat = 39.399872 # TODO: get the lat and lng for the town
lng = -8.224454
unit_village = "kW" # unit of the demand data

middle = df_sub.shape[1] // 2
df_sub_vill = df_sub.loc[:, df_sub.median().sort_values(ascending=False).index].iloc[:, middle:middle+5]
selected_cols =["MT_230", "MT_250", "MT_278"]
df_sub_vill = df_sub_vill[selected_cols] * 1e3 # convert to kW
village_acronnyms = {col: f'village_{i}' for i, col in enumerate(df_sub_vill.columns)}


store = pd.HDFStore(os.path.join(save_path, f"{spatial_scale}.h5"), mode='w') 
for temp_resolution in temp_resolutions:
    dfs = {}
    for key, value in village_acronnyms.items():
        df = df_sub_vill[key].to_frame(f'{value}_demand_{unit_village}')
        df = standardize_format(df, 'load', temp_resolution, value, unit_village)
        df = remove_non_positive_values(df, set_nan=True)
        train, val, test = split_train_val_test_datasets(df, train_begin, train_end, val_begin, val_end, test_begin, test_end)
        dfs[value] = df
        store.put(f'{value}/{temp_resolution}min/train_target', train, format='table')
        store.put(f'{value}/{temp_resolution}min/val_target', val, format='table')
        store.put(f'{value}/{temp_resolution}min/test_target', test, format='table')


        start_date = df.index[0].strftime("%Y-%m-%d")
        end_date = df.index[-1].strftime("%Y-%m-%d")
        df_weather = get_weather_data(lat, lng, start_date, end_date, variables=['temperature_2m'], keep_UTC=False).tz_localize(None)
        df_weather = standardize_format(df_weather, 'temperature', temp_resolution, value, "C")
        df_weather = df_weather.reindex(df.index).dropna() 
        dfs[value + 'weather'] = df_weather
        train_weather, val_weather, test_weather = split_train_val_test_datasets(df_weather, train_begin, train_end, val_begin, val_end, test_begin, test_end)

        store.put(f'{value}/{temp_resolution}min/train_cov', train_weather, format='table')
        store.put(f'{value}/{temp_resolution}min/val_cov', val_weather, format='table')
        store.put(f'{value}/{temp_resolution}min/test_cov', test_weather, format='table')
        

store.close() 
# Merge all the dataframes into one
df_village = pd.concat(dfs.values(), axis=1)
df_village.columns = dfs.keys()

Visualization

In [ ]:
location = 'village_0'

df= df_village

fig = go.Figure()
# Add the ground truth data to the left axis
fig.add_trace(go.Scatter(y=df[location], x=df.index, name = location,  yaxis="y1"))
fig.add_trace(go.Scatter(y=df[location+'weather'], x=df.index, name = 'Temp',  yaxis="y2"))

fig.update_layout(
    xaxis=dict(title="Time"),
    yaxis=dict(title=f"Power {unit_village}", side="left"),
    yaxis2=dict(title="Temperature [°C]", overlaying="y", side="right"),
)

Statistics

In [ ]:
calculate_stats_and_plot_hist(df_village[[col for col in df_village.columns if not 'weather' in col]])

## &#x2462; Building Data Genome 2 (BDG2) Data-Set

[![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.3887306.svg)](https://doi.org/10.5281/zenodo.3887306)


Note that the meter data has been cleaned found in the 'building-data-genome-project-2-v1.0\data\meters\cleaned\' folder. Furthermore, measurements are in kW unit, and the timezone is set to the local timezone. Also note that builds with the usage of 'Education' have been used because they exhibit the most complete data record.

Used for: 

* 4_neighborhood
* 5_building

----------
* Duration: 2 years
* Resolution: 60 minutes
* Power Level: 0-3000 W

### Data Selection

In [ ]:
df_gen = pd.read_csv(r'data\raw_data\building-data-genome-project-2-v1.0\data\meters\cleaned\electricity_cleaned.csv', index_col=0, parse_dates=True)
df_meta = pd.read_csv(r'data\raw_data\building-data-genome-project-2-v1.0\data\metadata\metadata.csv')


### 4_neighborhood

In [ ]:
df_meta.groupby('site_id')['primaryspaceusage'].value_counts()

In [ ]:
df_meta.groupby('site_id')['primaryspaceusage'].value_counts().to_frame().unstack().plot(kind='bar', figsize=(20,10))

In [ ]:
df_meta_neighborhood = df_meta.loc[
                                        (df_meta['site_id'] == 'Bull') 
                                         ]

df_meta_neighborhood.set_index('building_id', inplace=True)
cols_neighborhood = list(set(df_meta_neighborhood.index.to_list()) & set(df_gen.columns.to_list()))

In [ ]:
df_plot_neigh = df_gen[cols_neighborhood]
px.line(df_plot_neigh.sum(axis=1))

In [ ]:
df_neighborhoods_1 = pd.DataFrame()

neighborhoods = ['Hog', 'Bobcat', 'Bull']
for neighborhood in neighborhoods:
    df_meta_neighborhood = df_meta.loc[
                                        (df_meta['site_id'] == neighborhood) 
                                         ]
    df_meta_neighborhood.set_index('building_id', inplace=True)
    cols_neighborhood = list(set(df_meta_neighborhood.index.to_list()) & set(df_gen.columns.to_list()))
    df_plot_neigh = df_gen[cols_neighborhood].sum(axis=1).to_frame(f'{neighborhood}')
    df_neighborhoods_1 = pd.concat([df_neighborhoods_1, df_plot_neigh], axis=1)

In [ ]:
def remove_outliers(df, column, lower_percentile=0, upper_percentile=100):
    lower_threshold = df[column].quantile(lower_percentile / 100)
    upper_threshold = df[column].quantile(upper_percentile / 100)
    df_filtered = df[(df[column] >= lower_threshold) & (df[column] <= upper_threshold)]
    return df_filtered


In [ ]:
temp_resolutions = [60] # minutes
# Determined visually to meet criteria mentioned above
train_begin = "2016-01-01"
train_end = "2017-01-01"
val_begin = "2017-08-01"
val_end = "2017-09-01"
test_begin = "2017-12-01"
test_end = "2017-12-31"

unit = "kW" # unit of the demand data
spatial_scale = "4_neighborhood" # spatial scale of the data
arconyms_neighborhood = {col: f'neighborhood_{i}' for i, col in enumerate(neighborhoods)}

store = pd.HDFStore(os.path.join(save_path, f"{spatial_scale}.h5"), mode='w')
for temp_resolution in temp_resolutions:
    dfs = {}
    for key, value in arconyms_neighborhood.items():

        tf = TimezoneFinder()
        lat = df_meta.groupby('site_id')['lat'].mean().values[0]
        lng = df_meta.groupby('site_id')['lng'].mean().values[0]
        tz = tf.timezone_at(lng=lng, lat=lat)
        # power
        df = df_neighborhoods_1[key].to_frame(f'{value}_demand_{unit}')
        #df = df.tz_localize('UTC').tz_convert(tz).tz_localize(None) # the data is already in the correct timezone
        df = standardize_format(df, "load", temp_resolution, value, unit)
        #df = df.apply(lambda x: x.sub(x.groupby(x.index.date).transform("min"))) # to avoid hovering above 0 for some profiles
        df = remove_days(df, 0.1)
        df = remove_non_positive_values(df, set_nan=True)
        df = remove_outliers(df, df.columns[0], lower_percentile=0.05, upper_percentile=99.9)
        train, val, test = split_train_val_test_datasets(df, train_begin, train_end, val_begin, val_end, test_begin, test_end)
        dfs[value] = df

        store.put(f'{value}/{temp_resolution}min/train_target', train, format='table')
        store.put(f'{value}/{temp_resolution}min/val_target', val, format='table')
        store.put(f'{value}/{temp_resolution}min/test_target', test, format='table')

        # weather
        
        start_date = df.index[0].strftime("%Y-%m-%d")
        end_date = df.index[-1].strftime("%Y-%m-%d")
        df_weather = get_weather_data(lat, lng, start_date, end_date, variables=['temperature_2m'], keep_UTC=False).tz_localize(None)
        df_weather = standardize_format(df_weather, 'temperature', temp_resolution, value, "C")
        df_weather = df_weather.reindex(df.index).dropna() 
        dfs[value + 'weather'] = df_weather
        train_weather, val_weather, test_weather = split_train_val_test_datasets(df_weather, train_begin, train_end, val_begin, val_end, test_begin, test_end)

        store.put(f'{value}/{temp_resolution}min/train_cov', train_weather, format='table')
        store.put(f'{value}/{temp_resolution}min/val_cov', val_weather, format='table')
        store.put(f'{value}/{temp_resolution}min/test_cov', test_weather, format='table')
        

store.close()

# Merge all the dataframes into one for visualization
df_neighborhoods = pd.concat(dfs.values(), axis=1)
df_neighborhoods.columns = dfs.keys()

In [ ]:
location = 'neighborhood_0'

df= df_neighborhoods

fig = go.Figure()
# Add the ground truth data to the left axis
fig.add_trace(go.Scatter(y=df[location], x=df.index, name = location,  yaxis="y1"))
fig.add_trace(go.Scatter(y=df[location+'weather'], x=df.index, name = 'Temp',  yaxis="y2"))

fig.update_layout(
    xaxis=dict(title="Time"),
    yaxis=dict(title=f"Power {unit_village}", side="left"),
    yaxis2=dict(title="Temperature [°C]", overlaying="y", side="right"),
)

In [ ]:
calculate_stats_and_plot_hist(df_neighborhoods[[col for col in df_neighborhoods.columns if not 'weather' in col]])

### 5_building

In [ ]:
df_meta_building = df_meta.loc[
                                        (df_meta['primaryspaceusage'] == 'Education') 
                                         ]

df_meta_building.set_index('building_id', inplace=True)
cols_building = list(set(df_meta_building.index.to_list()) & set(df_gen.columns.to_list()))

In [ ]:
df_plot_gen = df_gen[cols_building]
# limits for max load of each building (we want rather small buildings)
max = 12
min = 0
df_plot_gen = df_plot_gen.loc[:, (df_plot_gen.max() < max) & (df_plot_gen.min() > min)]

In [ ]:
px.line(df_plot_gen, x=df_plot_gen.index, y=df_plot_gen.columns)

In [ ]:
# the following locations have been determined visually to meet the criteria mentioned above
buildings = ['Bear_education_Sandy', 'Bear_education_Millie', 'Cockatoo_education_Joel']
df_buildings = df_gen[buildings] * 1e3 # convert to W
df_meta_buildings = df_meta_building.loc[buildings]

In [ ]:
temp_resolutions = [60] # minutes
# Determined visually to meet criteria mentioned above
train_begin = "2016-01-01"
train_end = "2017-01-01"
val_begin = "2017-08-01"
val_end = "2017-09-01"
test_begin = "2017-12-01"
test_end = "2017-12-31"

unit = "W" # unit of the demand data
spatial_scale = "5_building" # spatial scale of the data
arconyms_building = {col: f'building_{i}' for i, col in enumerate(buildings)}

store = pd.HDFStore(os.path.join(save_path, f"{spatial_scale}.h5"), mode='w')
for temp_resolution in temp_resolutions:
    dfs = {}
    for key, value in arconyms_building.items():

        tf = TimezoneFinder()
        lat, lng = df_meta_buildings.loc[key, 'lat'], df_meta_buildings.loc[key, 'lng']
        tz = tf.timezone_at(lng=lng, lat=lat)
        # power
        df = df_buildings[key].to_frame(f'{value}_demand_{unit}')
        #df = df.tz_localize('UTC').tz_convert(tz).tz_localize(None)
        df = standardize_format(df, "load", temp_resolution, value, unit)
        df = df.apply(lambda x: x.sub(x.groupby(x.index.date).transform("min"))) # to avoid hovering above 0 for some profiles
        df = remove_days(df, 0.1)
        df = remove_non_positive_values(df, set_nan=True)
        df = remove_outliers(df, df.columns[0], lower_percentile=0.01, upper_percentile=100)

        train, val, test = split_train_val_test_datasets(df, train_begin, train_end, val_begin, val_end, test_begin, test_end)
        dfs[value] = df

        store.put(f'{value}/{temp_resolution}min/train_target', train, format='table')
        store.put(f'{value}/{temp_resolution}min/val_target', val, format='table')
        store.put(f'{value}/{temp_resolution}min/test_target', test, format='table')

        # weather
        
        start_date = df.index[0].strftime("%Y-%m-%d")
        end_date = df.index[-1].strftime("%Y-%m-%d")
        df_weather = get_weather_data(lat, lng, start_date, end_date, variables=['temperature_2m'], keep_UTC=False).tz_localize(None)
        df_weather = standardize_format(df_weather, 'temperature', temp_resolution, value, "C")
        df_weather = df_weather.reindex(df.index).dropna() 
        dfs[value + 'weather'] = df_weather
        train_weather, val_weather, test_weather = split_train_val_test_datasets(df_weather, train_begin, train_end, val_begin, val_end, test_begin, test_end)

        store.put(f'{value}/{temp_resolution}min/train_cov', train_weather, format='table')
        store.put(f'{value}/{temp_resolution}min/val_cov', val_weather, format='table')
        store.put(f'{value}/{temp_resolution}min/test_cov', test_weather, format='table')
        

store.close()

# Merge all the dataframes into one for visualization
df_buildings = pd.concat(dfs.values(), axis=1)
df_buildings.columns = dfs.keys()


In [ ]:
location = 'building_1'

df= df_buildings

fig = go.Figure()
# Add the ground truth data to the left axis
fig.add_trace(go.Scatter(y=df[location], x=df.index, name = location,  yaxis="y1"))
fig.add_trace(go.Scatter(y=df[location+'weather'], x=df.index, name = 'Temp',  yaxis="y2"))

fig.update_layout(
    xaxis=dict(title="Time"),
    yaxis=dict(title=f"Power {unit_village}", side="left"),
    yaxis2=dict(title="Temperature [°C]", overlaying="y", side="right"),
)

In [ ]:
calculate_stats_and_plot_hist(df_buildings[[col for col in df_buildings.columns if not 'weather' in col]])

### Checking what is in the saved files

In [ ]:

from utils import get_hdf_keys
# See what keys are in the h5py data file
locations_dict, resolutions_dict = get_hdf_keys(dir_path)

print("Locations: ", locations_dict)
print("Resolutions: ", resolutions_dict)

for scale, locations in locations_dict.items():
    if scale == '5_building.h5':
        for location in locations:
            df_train = pd.read_hdf(os.path.join(dir_path, scale), key=f'{location}/60min/train_target')
            df_val = pd.read_hdf(os.path.join(dir_path, scale), key=f'{location}/60min/val_target')
            df_test = pd.read_hdf(os.path.join(dir_path, scale), key=f'{location}/60min/test_target')
            fig = px.line(df_train, title=f'{scale}: {location}')
            fig.add_trace(px.line(df_val, title='Validation Set').data[0])
            fig.add_trace(px.line(df_test, title='Test Set').data[0])
            fig.show()



# Datasets below are not used in the paper, but might be useful.

## &#x2463; 'WPuQ' - Household data (Germany)
Note that since this data was recorded in a village in Germany, the aggregate of the data will be used as a neighborhood demand 

[![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.5642902.svg)](https://doi.org/10.5281/zenodo.5642902)

Used for: 

* 4_neighborhood
* 5_household




### Data Selection

In [ ]:
#h5py extraction; only for the german data; cleaning code starts in the next cell
resolution = 15
dfs_load = []
dfs_temperature = []
for year in [2018, 2019, 2020]:
    # Load
    filename = os.path.join("data","raw_data","households_GER","power",f"{year}_data_{resolution}min.hdf5")
    f = h5py.File(filename)
    group_no_pv = f["NO_PV"] #Only regard those profiles that are not mixed with PV generation
    dfs = {}

    for key in group_no_pv.keys():
        table = group_no_pv[key]["HOUSEHOLD"]
        table = table["table"][:]
        df = pd.DataFrame(table).dropna().set_index("index")[["P_TOT"]]
        df.index = pd.to_datetime(df.index, unit = "s")
        dfs[key] = df

    df_load = pd.concat(list(dfs.values()), axis=1)
    df_load.columns = list(dfs.keys())
    dfs_load.append(df_load)

    # Weather is available but includes a lot of missing values, so we will use the API instead (next cell) 
    filename_weather = os.path.join("data","raw_data","households_GER","weather",f"{year}_weather.hdf5")

    f_weather = h5py.File(filename_weather)
    df_temp = pd.DataFrame(f_weather['WEATHER_SERVICE']['IN']['WEATHER_TEMPERATURE_TOTAL']['table'][:]).set_index("index").dropna()
    df_temp.index = pd.to_datetime(df_temp.index, unit = "ns")
    dfs_temperature.append(df_temp)
    

df_load_final = pd.concat(dfs_load, axis = 0)
df_temp_final = pd.concat(dfs_temperature, axis = 0)
# subtract the minimum value of each day because some profiles are 'levitating' above 0


In [ ]:
# cleaning code starts here

temp_resolutions = [15, 60] # minutes
# Determined visually to meet criteria mentioned above
train_begin = "2018-08-01"
train_end = "2019-08-01"
val_begin = "2020-01-01"
val_end = "2020-02-01"
test_begin = "2019-08-01"
test_end = "2019-09-01"



# Determined visually to meet criteria mentioned above
columns_neighborhood = ['SFH3', 'SFH4', 'SFH5', 'SFH9', 'SFH10', 'SFH12', 'SFH16','SFH18','SFH19', 'SFH21',
                        'SFH22', 'SFH23', 'SFH27', 'SFH28', 'SFH29', 'SFH30', 'SFH31',
                        'SFH32', 'SFH36', 'SFH38']

# for later use in the notebook
df_neighborhood_1 = df_load_final[columns_neighborhood].sum(axis=1).to_frame("demand") / 1e3 # convert to kW


### Data Cleaning & Saving

In [ ]:
# 5_household
unit = "W" # unit of the demand data
spatial_scale = "5_household" # spatial scale of the data
lat = 52.266666
lng = 10.516667
columns_household = ["SFH4", "SFH36", "SFH12"] # selected columns through visual inspection (not performed here)
arconyms_household = {col: f'household_{i}' for i, col in enumerate(columns_household)}

store = pd.HDFStore(os.path.join(save_path, f"{spatial_scale}.h5"), mode='w')
for temp_resolution in temp_resolutions:
    dfs = {}
    for key, value in arconyms_household.items():

        tf = TimezoneFinder()
        tz = tf.timezone_at(lng=lng, lat=lat)
        # power
        df = df_load_final[key].to_frame(f'{value}_demand_{unit}')
        df = df.tz_localize('UTC').tz_convert(tz).tz_localize(None)
        df = standardize_format(df, "load", temp_resolution, value, unit)
        df = df.apply(lambda x: x.sub(x.groupby(x.index.date).transform("min"))) # to avoid hovering above 0 for some profiles
        df = remove_days(df, 0.1)
        df = remove_non_positive_values(df)
        train, val, test = split_train_val_test_datasets(df, train_begin, train_end, val_begin, val_end, test_begin, test_end)
        dfs[value] = df

        store.put(f'{value}/{temp_resolution}min/train_target', train, format='table')
        store.put(f'{value}/{temp_resolution}min/val_target', val, format='table')
        store.put(f'{value}/{temp_resolution}min/test_target', test, format='table')

        # weather
        
        start_date = df.index[0].strftime("%Y-%m-%d")
        end_date = df.index[-1].strftime("%Y-%m-%d")
        df_weather = get_weather_data(lat, lng, start_date, end_date, variables=['temperature_2m'], keep_UTC=False).tz_localize(None)
        df_weather = standardize_format(df_weather, 'temperature', temp_resolution, value, "C")
        df_weather = df_weather.reindex(df.index).dropna() 
        dfs[value + 'weather'] = df_weather
        train_weather, val_weather, test_weather = split_train_val_test_datasets(df_weather, train_begin, train_end, val_begin, val_end, test_begin, test_end)

        store.put(f'{value}/{temp_resolution}min/train_cov', train_weather, format='table')
        store.put(f'{value}/{temp_resolution}min/val_cov', val_weather, format='table')
        store.put(f'{value}/{temp_resolution}min/test_cov', test_weather, format='table')
        

store.close()

# Merge all the dataframes into one for visualization
df_households = pd.concat(dfs.values(), axis=1)

# save the weather data for the neighborhood saving (see below)
df_neighborhood_1_weather = df_households.filter(like='temp').iloc[:, 0].to_frame("temperature")


In [ ]:
df_households.columns = dfs.keys()

### Visualization

In [ ]:
df = df_households.resample("4H").mean()

location = 'household_0'

fig = go.Figure()
# Add the ground truth data to the left axis
fig.add_trace(go.Scatter(y=df[location], x=df.index, name = location,  yaxis="y1"))
fig.add_trace(go.Scatter(y=df[location+'weather'], x=df.index, name = 'Temp',  yaxis="y2"))

fig.update_layout(
    xaxis=dict(title="Time"),
    yaxis=dict(title=f"Power", side="left"),
    yaxis2=dict(title="Temperature [°C]", overlaying="y", side="right"),
)


## &#x2464; 'MFRED' Apartment data - United States of America
[![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.3690240.svg)](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/X9MIDJ)

Used for: 

* 6_apartments

----------
* Duration: 1 year (Note: While this does not strictly meet the requirements, we have included this dataset due to its high quality. To achieve two years of data, we select two aparments that have an extremely similar yearly profile (measured by ) and concatenate these two as one timeseries)
* Resolution: 15 minutes
* Power Level: 0-3000 W

### Data Read in

In [ ]:
dir_path = os.path.join('data','raw_data','apartments_US','MFRED_Aggregates_15min_2019Q1-Q4.csv')

spatial_scale = '6_apartment'
unit_apartment = 'W'
lat, lng = 40.776676, 73.971321 # New York City
temp_resolutions = [5, 15, 60]

df_ap = pd.read_csv(dir_path, index_col =0 , parse_dates = True).filter(regex=r'^.*kW$')*1000
df_ap = df_ap.iloc[:, 1:] # exluding the aggregated load series
df_ap.columns = [col.split('_')[0] for col in df_ap.columns] # renaming the columns since we converted to Watts above
arconyms_apartment = {col: col.split("G")[1] for col in df_ap.columns}


### Visualization

In [ ]:
df_plot = df_ap.resample("4H").mean()

fig = px.line(df_plot)
fig.update_layout(title='Apartment Power Demand', xaxis_title='Time', yaxis_title='Power (W)')

### Special Processing: Concatenating similar apartment profiles

As mentioned above, this dataset does not strictly meet the requirements. However, we have included this dataset due to its high measurement quality. To achieve two years of data (1 for training, the other for testing), we select two aparments that have a similar yearly profile (measured by [dynamic timewarping (DTW)](https://en.wikipedia.org/wiki/Dynamic_time_warping)) and concatenate these two as one timeseries

In [ ]:
# first we need to scale the data to cancel the effect of capacities (e.g. two apartments might have a similar behaviour but different appliances)
scaler = MinMaxScaler()
df_plot_scaled = pd.DataFrame(scaler.fit_transform(df_plot), columns=df_plot.columns, index=df_plot.index)
df_plot_scaled = df_plot_scaled.fillna(method = "bfill", limit = 4).dropna()

In [ ]:
# we will calculate the DTW on the df_plot dataframe which is resampled to 4 hours. This saves a lot of time,
# while still preserving the intra-daily characteristics of the data

dtw_matrix = dtw_distance_matrix(df_plot_scaled)

# Then, get the upper triangle of the matrix (excluding the diagonal) to avoid duplicates
dtw_upper_triangle = dtw_matrix.where(np.triu(np.ones(dtw_matrix.shape), k=1).astype(np.bool))

# Sort the values in ascending order and get the indices of the three smallest values
smallest_indices = dtw_upper_triangle.unstack().sort_values().index.tolist()


In [ ]:
# Print the column pairs with the smallest DTW distances
for pair in smallest_indices[:8]:
    print(f"Columns '{pair[0]}' and '{pair[1]}' have a DTW distance of {dtw_matrix.loc[pair[0], pair[1]]}.")

In [ ]:
# Plotting the two combinations with the smallest DTW distance
for i in range(5):
    fig = px.line(df_plot_scaled[list(smallest_indices[i])], title=f"Apartment Power Demand {i}")
    fig.update_layout(xaxis_title='Time', yaxis_title='Power (W)')
    fig.show()

In [ ]:
df_ap_to_concat_1 = concat_and_scale(df_ap, smallest_indices[0]) # this is the pair with the smallest DTW distance
df_ap_to_concat_2 = concat_and_scale(df_ap, smallest_indices[4]) # these are the two pairs with the second-smallest DTW distance that have no overlap with the one above
df_ap_to_concat_3 = concat_and_scale(df_ap, smallest_indices[5]) # these are the two pairs with the second-smallest DTW distance that have no overlap with the one above

### Visualizing the two pairs of apartments with the smallest DTW distance

In [ ]:
fig = px.line(df_ap_to_concat_1, title=f"Apartment Power Demand 1")
fig.update_layout(xaxis_title='Time', yaxis_title='Power (W)')
fig.show()

fig_2 = px.line(df_ap_to_concat_2, title=f"Apartment Power Demand 2")
fig_2.update_layout(xaxis_title='Time', yaxis_title='Power (W)')
fig_2.show()


In [ ]:
df_apartments = pd.concat([df_ap_to_concat_1, df_ap_to_concat_2, df_ap_to_concat_3], axis=1)
# as these are now hybrids of 6 apartments, we need to rename the columns
df_apartments.columns = [f"apartment_{i}" for i in range(1, len(df_apartments.columns)+1)]

### The cleaning and saving (Like in the other spatial scales)

In [ ]:
# Determined visually to meet criteria mentioned above
train_begin = "2019-01-01"
train_end = "2020-01-01"
val_begin = "2020-01-01"
val_end = "2020-02-01"
test_begin = "2020-08-01"
test_end = "2020-09-01"

# 4_neighborhood
spatial_scale = "6_apartment" # spatial scale of the data
unit = "W" # unit of the demand data
lat, lng = 40.776676, 73.971321 # New York City

arconyms_neighborhood = {col: f'apartment_{i}' for i, col in enumerate(df_apartments.columns)}

store = pd.HDFStore(os.path.join(save_path, f"{spatial_scale}.h5"), mode='w')
for temp_resolution in temp_resolutions:
    dfs = {}
    for key, value in arconyms_neighborhood.items():
        df = df_apartments[key].to_frame(f'{value}_demand_{unit}')
        df = standardize_format(df, "load", temp_resolution, value, unit)
        df = df.apply(lambda x: x.sub(x.groupby(x.index.date).transform("min"))) # to avoid hovering above 0 for some profiles
        df = remove_non_positive_values(df)
        train, val, test = split_train_val_test_datasets(df, train_begin, train_end, val_begin, val_end, test_begin, test_end)
        dfs[value] = df

        store.put(f'{value}/{temp_resolution}min/train_target', train, format='table')
        store.put(f'{value}/{temp_resolution}min/val_target', val, format='table')
        store.put(f'{value}/{temp_resolution}min/test_target', test, format='table')

        start_date = df.index[0].strftime("%Y-%m-%d")
        end_date = df.index[-1].strftime("%Y-%m-%d")
        df_weather = get_weather_data(lat, lng, start_date, end_date, variables=['temperature_2m'], keep_UTC=False).tz_localize(None)
        df_weather = standardize_format(df_weather, 'temperature', temp_resolution, value, "C")
        df_weather = df_weather.reindex(df.index).dropna() 
        dfs[value + 'weather'] = df_weather
        train_weather, val_weather, test_weather = split_train_val_test_datasets(df_weather, train_begin, train_end, val_begin, val_end, test_begin, test_end)

        store.put(f'{value}/{temp_resolution}min/train_cov', train_weather, format='table')
        store.put(f'{value}/{temp_resolution}min/val_cov', val_weather, format='table')
        store.put(f'{value}/{temp_resolution}min/test_cov', test_weather, format='table')



store.close()


# Debugging

In [3]:
get_hdf_keys(CLEAN_DATA_DIR)

({'5_building.h5': ['building_0', 'building_1', 'building_2'],
  '4_neighborhood.h5': ['neighborhood_0', 'neighborhood_1', 'neighborhood_2'],
  '3_village.h5': ['village_0', 'village_1', 'village_2'],
  '2_town.h5': ['town_0', 'town_1', 'town_2'],
  '1_county.h5': ['Los_Angeles', 'New_York', 'Sacramento']},
 {'5_building.h5': ['60min'],
  '4_neighborhood.h5': ['60min'],
  '3_village.h5': ['15min', '60min'],
  '2_town.h5': ['15min', '60min'],
  '1_county.h5': ['60min']})

In [6]:


# Loading Data
df_train = pd.read_hdf(
    os.path.join(CLEAN_DATA_DIR, f"1_county.h5"),
    key=f"Los_Angeles/60min/train_target",
)

In [4]:
from utils.data_utils import plot_location_splits


In [10]:
plot_location_splits(CLEAN_DATA_DIR, 0, 0, show="both");